# Solve Using QAOA

In [2]:
import pandas as pd
from qiskit.circuit.library import TwoLocal
import numpy as np
import datetime as dt
from qiskit_algorithms import QAOA
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer,WarmStartQAOAOptimizer, CplexOptimizer, CobylaOptimizer
from qiskit.primitives import Sampler
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils.algorithm_globals import algorithm_globals

Warm-QAOA faster but giving error when using high value

Rules for preprocessing::

Flights to not consider in case of delayed flights:

At the departure Airport-
1. Do not consider the flights which depart x hrs before the initial departure time. (decide on the x)
2. Do not consider the flights which depart after the delayed arrival time

At the arrival Airport-
3. Do not consider the flights which arrive after the delayed arrival time

4. Do not consider the flights which fly at roughly the same time window as the affected flight but their departure and arrival airports are different.
eg. Affected flight is MUM-DEL (18:00 - 21:00)
     Do not consider BNG - PNQ (18:05 - 20:35)

5. Take no flight arriving to departing airport of impacted flight 
7. Take no flight departing from arrival airport of impacted flight

7. None of the flights on the path should be delayed or cancelled.


In [12]:


class QuantumSolver:
    df = None  # INV.csv
    highval = 1000000  # used for G (neglecting some flights)
    inv_id: str  # stores the affected flight inventory id
    flight = None  # affected flight row
    FullList:dict
    Q = A = B = Nb = Na = G = None  # matrices to be used in Quadratic Program

    def __init__(self, inv_id, FullList, inFile):
        """
            Input : Corresponding inventory id of affected flight from the provided datafile
            Creates a list of relevant flights, initializes matrices, and calls the QuantumSolve() function
        """

        self.df = pd.read_csv(inFile)

        startTime = dt.datetime.now()
        self.inv_id = inv_id
        self.FullList=FullList

        self.lst = self.__preProcess()  # list of relevant flights
        if len(self.lst) == 0:  # no relevant flights present
            raise Exception('No alternate flights')
        print(len(self.lst), "Feasible Flights\n")
        
        self.length = len(self.lst)
        self.length=min(self.length,15)
        file=open("Selected Flights","w")
        for i in self.lst[:self.length]:
            file.write(str(i)+"\n\n")
        file.close()

        # all matrices
        self.Q = np.zeros((self.length, self.length))
        self.A, self.B, self.Nb, self.G, self.Na = np.zeros_like(self.Q), np.zeros_like(self.Q), np.zeros_like(self.Q), np.zeros_like(self.Q), np.zeros_like(self.Q)

        print("\nAffected Flight\n")
        print(self.flight)

        ans = self.quantumSolve()  # invoke the quantum solve function
        TimeTaken=(dt.datetime.now() - startTime)

        for i in range(len(ans)):
            print("Solution",i,end="\n\n")
            print(ans[i])
            print("\n")

        print(TimeTaken)

    def __diff(self, date1, time1, date2, time2):
        """
            returns difference in time of date2-date1
        """
        dt1 = dt.datetime.strptime(date1 + " " + time1, "%m/%d/%Y %H:%M")
        dt2 = dt.datetime.strptime(date2 + " " + time2, '%m/%d/%Y %H:%M')
        difference = dt2 - dt1
        return int(difference.total_seconds() // 60)

    def __preProcess(self) -> list:
        """
            Creates a list of relevant flights which have sensible departure and arrival times
        """
        index = self.df.loc[self.df["InventoryId"] == self.inv_id].index[0]  # get the index from the dataset
        self.flight = self.df.loc[index]  # affected flight
        list_of_feasible_flights = []
        currFlTime=self.__diff(date1=self.flight["DepartureDate"], time1=self.flight["DepartureTime"], date2=self.flight["ArrivalDate"],
                              time2=self.flight["ArrivalTime"])

        for i in range(len(self.df)):
            data = self.df.loc[i]
            if index == i:
                continue
            fromlst=self.FullList.get(data["InventoryId"])
            if fromlst:
                if fromlst=="Cancelled":
                    continue
            if data["ArrivalAirport"] == self.flight["DepartureAirport"] or data["DepartureAirport"]==self.flight["ArrivalAirport"]:
                continue
            timebtwArr_Curdep = self.__diff(date1=self.flight["DepartureDate"], time1=self.flight["DepartureTime"], date2=data["ArrivalDate"],
                             time2=data["ArrivalTime"])
            timebtwdep_CurrDep = self.__diff(date1=self.flight["DepartureDate"], time1=self.flight["DepartureTime"], date2=data["DepartureDate"],
                              time2=data["DepartureTime"])
            flightTime=self.__diff(date1=data["DepartureDate"], time1=data["DepartureTime"], date2=data["ArrivalDate"],
                              time2=data["ArrivalTime"])
            
            if timebtwArr_Curdep < 60 or timebtwdep_CurrDep > 72 * 60:   # rule 1,2,3  # following the Minimum Ground Time and Maximum Connecting Time
                continue
            elif flightTime > currFlTime and data["DepartureAirport"] != self.flight["DepartureAirport"] \
                    and data["ArrivalAirport"] != self.flight["ArrivalAirport"]   :  # rule 4
                continue
            else:
                list_of_feasible_flights.append(data)

        return list_of_feasible_flights

    def __run(self):
        """
            Creates the Matrices required for the Quadratic Program
        """
        self.graph = dict()  # to keep a tally of unique flights

        # Nodes
        for i in range(self.length):
            data = self.lst[i]  # a single flight

            if data["InventoryId"] not in self.graph:
                self.graph[data["InventoryId"]] = (
                    data["DepartureAirport"],
                    data["ArrivalAirport"],
                    data["DepartureDate"],
                    data["DepartureTime"],
                    data["ArrivalTime"],
                    data["ArrivalDate"]
                )

                # Q Matrix measures the total time taken by the flight (Arr - Dep of the flight)
                self.Q[i, i] = self.__diff(data["DepartureDate"], data["DepartureTime"], data["ArrivalDate"],
                                           data["ArrivalTime"]) * 100

                # A Matrix measures if the current flight departs from the same airport as Affected flight
                # Nb Matrix measures the neighbouring flight connectivity(by deparuture)
                # Na Matrix measures the neighbouring flight connectivity(by arrival)
                # G Matrix measures the ground time spent by a flight (next departure - arrival)
                if data["DepartureAirport"] == self.flight["DepartureAirport"]:
                    self.A[i, i] = 1
                    self.Nb[i, i] = 1
                    self.G[i, i] = self.__diff(self.flight["DepartureDate"], self.flight["DepartureTime"],
                                               data["DepartureDate"], data["DepartureTime"]) * 100
                else:
                    self.A[i, i] = 0
                if data["ArrivalAirport"] == self.flight["ArrivalAirport"]:
                    self.Na[i, i]= 1
                    self.B[i, i] = 1
                else:
                    self.B[i, i]= 0

                # Edges
                for j in range(self.length):
                    fl2 = self.lst[j]
                    if self.Nb[i, i] == 1 and i != j:
                        self.G[i, j] = 0
                    else:
                        if data["InventoryId"] == fl2["InventoryId"]:
                            ti = self.__diff(self.flight["DepartureDate"], self.flight["DepartureTime"],
                                             fl2["DepartureDate"], fl2["DepartureTime"]) * 100
                            self.G[i, j] = ti
                        else:
                            if data["DepartureAirport"] == fl2["ArrivalAirport"]:
                                self.Nb[j, i] = 1
                                ti = int(self.__diff(fl2["ArrivalDate"], fl2["ArrivalTime"], data["DepartureDate"],
                                                     data[
                                                         "DepartureTime"]) * 100)  # minutes(fl2["DepartureTime"]) - minutes(data["ArrivalTime"])
                                if ti < 6000 or ti > 72000:
                                    self.G[i, j] = 0
                                else:
                                    self.G[i, j] = ti
                            else:
                                self.Nb[j, i] = 0
                                self.G[i, j] = 0
                            if data["ArrivalAirport"] == fl2["DepartureAirport"]:
                                self.Na[i,j]=1
                            else:
                                self.Na[i,j]=1
        self.G = self.G.astype(int)
        # print("Q:\n ", self.Q)
        # print("G:\n ", self.G)
        # print("A:\n ", self.A)
        # print("B:\n ", self.B)
        # print("Nb:\n ", self.Nb)
        # print("Na:\n ", self.Na)

    def QAOA_algo(self) -> list:
        """
        Main Function of the Class
        Quadratic Program : 1. Objective Function - Q
                            2. Linear Constraints - A & B
                            3. Quadratic Constraints - N & G
        Why QUBO - Each flight has two options - Either it can be in the path or not
        :return: list of alternate flights as
        """

        qp = QuadraticProgram("flights")
        F = self.Q + self.G + self.highval*self.Nb + self.highval*self.Na # quadratic form matrix
        F = F.astype(int)
        # print(F)

        qp.minimize(quadratic=F)  # matrices fed into the quadratic program
        qp.binary_var_list(self.length)
        qp.linear_constraint(np.ones(self.length, ), ">", 1)
        qp.linear_constraint(self.A.diagonal(), "=", 1)
        qp.linear_constraint(self.B.diagonal(), "=", 1)
        # print(qp.prettyprint())

        qp = QuadraticProgramToQubo().convert(qp)
        # qp.quadratic_constraint(quadratic=self.Nb,linear=-1*np.ones((self.length,)),sense="=",rhs=0)
        print(qp.prettyprint())

        # qubitOp, offset = qp.to_ising()  # conversion into Ising Problem

        two = TwoLocal(self.length, 'rx', 'cx', 'linear', reps=2, insert_barriers=True)  # an ansatz circuit

        # start = dt.datetime.now()
        algorithm_globals.random_seed = 12345

        qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA())  # a qaoa instance of the quadratic program
        qaoa_mes.ansatz = two
        # ws_qaoa = WarmStartQAOAOptimizer(pre_solver=CobylaOptimizer(),relax_for_pre_solver=True,qaoa=qaoa_mes,epsilon=0.0)
        ws_qaoa = WarmStartQAOAOptimizer(pre_solver=CplexOptimizer(), relax_for_pre_solver=False, qaoa=qaoa_mes,
                                     epsilon=0.0)
        # qaoa = MinimumEigenOptimizer(qaoa_mes)
        ws_qaoa_result = ws_qaoa.solve(qp)
        print(ws_qaoa_result.prettyprint())

        ans = list(ws_qaoa_result.variables_dict.values())
        # print(dt.datetime.now() - start)

        return ans

    def quantumSolve(self) -> list:
        """
        Linker between all working functions
        :return: list of suitable alternative flights
        """
        total = []
        self.__run()  # we will get the Q A B N G matrices initialised now
        prevAns=[]
        # while True:
        ans=self.QAOA_algo()
            # if ans == prevAns:
            #     break
            # prevAns=ans
        total.append([self.__postProcess(ans[:self.length])])
        return total

    def __postProcess(self, bitString: list) -> list:
        """
        Convert the output from QAOA_algo() function into a suitable format
        :param bitString:
        :return: list of flights corresponding to the bitString
        """
        flights = []
        isStarting=False;isEnding=False # TO check if path is complete (If not complete do no add this path)
        for i in range(len(bitString)):
            if bitString[i]==1:
                flights.append(self.lst[i])
                if self.lst[i]["DepartureAirport"]==self.flight["DepartureAirport"]:
                    isStarting=True

                if self.lst[i]["ArrivalAirport"]==self.flight["ArrivalAirport"]:
                    isEnding=True
        if not isEnding or not isStarting:
            return []
        for i in range(len(bitString)):
            if bitString[i]==0:
                continue
            for j in range(len(bitString)):
                if bitString[j] == 1:
                    self.Q[i,j]=self.highval
        return flights

In [13]:
INVFile=["../INV.csv","../sampleData(toBeDeleted)/INV.csv","../sampleData(toBeDeleted)/INV copy.csv",
         "../sampleData(toBeDeleted)/INV copy 2.csv","../sampleData(toBeDeleted)/INV copy 3.csv",
         "../sampleData(toBeDeleted)/INV copy 4.csv","../sampleData(toBeDeleted)/INV copy 5.csv"]

In [14]:
# QuantumSolver("INV-ZZ-1875559",INVFile[0])
# QuantumSolver("INV-ZZ-5515021",INVFile[0])
FullList={"INV-ZZ-5480266":"Cancelled","INV-ZZ-2085988":"Cancelled","INV-ZZ-2640504":"Cancelled","INV-ZZ-5480266":"Delayed"}

QuantumSolver("INV-ZZ-2602588",FullList,INVFile[0])

23 Feasible Flights


Affected Flight

InventoryId          INV-ZZ-2602588
ScheduleId           SCH-ZZ-5237867
AircraftType        Airbus A320-200
DepartureDate            12/25/2023
ArrivalDate              12/25/2023
DepartureAirport                COK
ArrivalAirport                  CCU
DepartureTime                 10:23
ArrivalTime                   20:48
Name: 345, dtype: object
Problem name: flights

Minimize
  195159601*c0@int_slack@0^2 + 780638404*c0@int_slack@0*c0@int_slack@1
  + 1561276808*c0@int_slack@0*c0@int_slack@2
  + 2732234414*c0@int_slack@0*c0@int_slack@3 + 780638404*c0@int_slack@1^2
  + 3122553616*c0@int_slack@1*c0@int_slack@2
  + 5464468828*c0@int_slack@1*c0@int_slack@3 + 3122553616*c0@int_slack@2^2
  + 10928937656.0*c0@int_slack@2*c0@int_slack@3 + 9562820449*c0@int_slack@3^2
  - 390319202*x0*c0@int_slack@0 - 780638404*x0*c0@int_slack@1
  - 1561276808*x0*c0@int_slack@2 - 2732234414*x0*c0@int_slack@3 + 195518001*x0^2
  + 392319202*x0*x1 + 391319202*x0*x10 + 39131920